In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

     |████████████████████████████████| 6.4 MB 12.7 MB/s 
     |████████████████████████████████| 63 kB 819 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 2.4 MB/s 


In [351]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def read_txt_list():
  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[: , 1:]        
    data = np.array(data)
    llst = [0, 1, 2, 3, 5]
    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:
           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [3]:
def download_data(symbols,periodd,intervall):
  
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100

    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 12:
             data.to_csv(f"/content/data/{symbol}.csv")
             
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     data = np.array(unattached_dfs[0])
     for i in unattached_dfs[1:]:
           data = np.append(data, np.array(i), axis=0)
        
     unattached_dfs = []
  
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [ ]:
symbols = get_crypto_syms()
#symbols = read_txt_list()
#symbols = read_syms_from_txt()
#symbols = ["btc-usd","eth-usd","trx-usd","xrp-usd","ltc-usd"]

print(f"Symbols : {len(symbols)}")
download_data(symbols,"10d","2m")
folder_name = extract_data(100)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 
Files In Data : 1419
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120  
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160  
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200  
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 

In [ ]:
model = Sequential()



model.add(Dense(512, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))


opt = tf.keras.optimizers.Adamax()

model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xTrain,yTrain,epochs=30,batch_size=1000,validation_data=(xTest,yTest),shuffle=False)

In [416]:
data = yf.download("trx-usd",period="1d",interval="1m")
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-04-26 00:00:00+00:00,0.065323,0.065323,0.065323,0.065323,0.065323,0
2022-04-26 00:02:00+00:00,0.065303,0.065303,0.065303,0.065303,0.065303,0
2022-04-26 00:04:00+00:00,0.065318,0.065318,0.065318,0.065318,0.065318,0
2022-04-26 00:06:00+00:00,0.065294,0.065294,0.065294,0.065294,0.065294,0
2022-04-26 00:07:00+00:00,0.065238,0.065238,0.065238,0.065238,0.065238,362752
...,...,...,...,...,...,...
2022-04-26 18:08:00+00:00,0.062616,0.062616,0.062616,0.062616,0.062616,0
2022-04-26 18:10:00+00:00,0.062645,0.062645,0.062645,0.062645,0.062645,0
2022-04-26 18:11:00+00:00,0.062630,0.062630,0.062630,0.062630,0.062630,0


In [386]:
def process_for_prediction(data):
    i = -1
    row = []
    if len(pd.DataFrame(data).columns) == 7:
      data = data.iloc[: , 1:]        
    data = np.array(data)
    llst = [0, 1, 2, 3, 5]
    grow = []
    srow = []
    for j in llst:
           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

    for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        
    arr = np.array(grow).flatten()
    row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)
def make_prediction(symbol,timeframe):
  if timeframe == "5m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))    
  if timeframe == "15m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1h":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1d":
    data = yf.download(symbol,period="20d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1wk":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "2m":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "30m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "5d":
    data = yf.download(symbol,period="5mo",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "90m":
    data = yf.download(symbol,period="5d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "3mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  

In [410]:
make_prediction("trx-usd","1m")

[*********************100%***********************]  1 of 1 completed


array([[3.5567891e-07, 9.9999666e-01]], dtype=float32)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
